# Evaluate Generated Code
This notebook loads the code generated by an LLM and uses the modified human-eval package to evaluate how well the code was generated by computing the Pass@1 score. Pass@5 and Pass@10 scores can also be computed.
* [Original package](https://github.com/openai/human-eval) - evaluates code generaion models only on the [HumanEval](https://huggingface.co/datasets/openai/openai_humaneval) dataset.
* [My modifications](https://github.com/agnedil/code-generation/tree/main/modified-openai-human-eval-code) - I made an extensive addition to enable the evaluation on three more datasets: [MBPP](https://huggingface.co/datasets/google-research-datasets/mbpp), [LBPP](https://huggingface.co/datasets/CohereForAI/lbpp), and [Big Code Benchmark](https://huggingface.co/datasets/bigcode/bigcodebench)

__This version walks over the entire directory with all models and evaluated them in bulk, populating the dataframe with pass@1 scores for all models / prompt types / cleaning modes__.

The models should be evaluated in a specific order. This is required to populate the dataframe with results in the correct order. The order of prompts should be as follows:
* prompt_basic
* prompt
* prompt_full

Within each prompt, the order of cleaning modes should be as follows:
* raw
* cleaned_partially
* cleaned_fully
* cleaned_fully_light

The code in Section 2 enforces this order for prompts. The code in Section 3 enforces it for cleaning modes.

In [1]:
import os
import numpy as np
import pandas as pd
from human_eval.evaluation import evaluate_functional_correctness
from human_eval.data import stream_jsonl, write_jsonl
from datasets import load_dataset
from collections import defaultdict
from helpers import get_code_in_fences, clean_code, clean_code_light, read_problems

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
%load_ext autoreload
%autoreload 2

## 1. Load dataset

In [2]:
# select dataset
idx = 2
dataset_names = ['human_eval', 'big_code', 'lbpp', 'mbpp']
dataset_name  = dataset_names[idx]
mode          = dataset_names[idx]
print('Dataset:      ', dataset_name, '\nCleaning mode:', mode)

Dataset:       lbpp 
Cleaning mode: lbpp


In [3]:
# read dataset for 'human_eval' and 'big_code' (func signature needed)
# no need to load for 'lbpp' and 'mbpp'
if dataset_name == 'human_eval':
    from datasets import load_dataset
    dataset = load_dataset("openai/openai_humaneval")
    tasks   = {i['task_id']:i for i in dataset['test']}
elif dataset_name == 'big_code':
    file = 'data/Big_Code_Bench_Test.jsonl.gz'
    tasks = read_problems(file)
    
if dataset_name in ['human_eval', 'big_code',]:    
    print(f'Type of tasks: {type(tasks)}\n')
    counter = 0
    for k,v in tasks.items():
        if counter == 2:
            break
        print(f'task_id: {k}', type(k))
        for k2, v2 in v.items():
            print(f'\n{k2}:\n{v2}')
        print('\n' + '='*100 + '\n')
        counter += 1

In [4]:
if dataset_name == 'human_eval':
    for task_id in ['HumanEval/1', 'HumanEval/10']:
        print(task_id, '\n', tasks[task_id]['prompt'], sep='')
elif dataset_name == 'big_code':
    for task_id in ['BigCodeBench/1', 'BigCodeBench/10']:
        print(task_id, '\n', tasks[task_id]['prompt'], sep='')

## 2. Get filepaths

In [5]:
# select prompt type used in filenames: complete_code OR complete_task
idx = 1
objectives = ['code', 'task']
objective  = objectives[idx]
print('Objective: ', objective)
prompt_basic_key  = f'complete_{objective}_prompt_basic'
prompt_medium_key = f'complete_{objective}_prompt'
prompt_full_key   = f'complete_{objective}_prompt_full'

# models to be evaluated
model_names = [
    'phixtral-2x2',
    'Solar-10.7B',
    'Llama-3.1-8B',
    'codegemma-7b-it',
    'deepseek-coder-6.7b',
    'OpenCodeInterpreter-DS-6.7B',
    'Artigenz-Coder-DS-6.7B',
    'CodeQwen1.5-7B-Chat',
    'Nxcode-CQ-7B-orpo',
    #'phixtral-4x2',
    #'mistral_7b',
    #'mistral_3b',
    #'mistral_8B',
    #'nemo',
    #'codestral',
]
print('Num models:', len(model_names))

Objective:  task
Num models: 9


In [6]:
# group files by model
wdir = 'logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087'
jsonl_files = defaultdict(list)

for root, dirs, files in os.walk(wdir):
    for filename in files:
        if filename.endswith('.jsonl'):
            model_name = [name for name in model_names if name in filename]
            assert len(model_name)==1, f'None or too many model names in file {filename}'
            filepath = os.path.join(root, filename)            
            jsonl_files[model_name[0]].append(filepath)

print("Models with JSONL files found:")
print(len(jsonl_files))
for model, files in jsonl_files.items():
    print(model)
    print('Num files:', len(files))
    print('\n'.join(files), '\n')

Models with JSONL files found:
9
phixtral-2x2
Num files: 3
logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_basic_temperature1.0_topP0.87_completions_20250620_110343_5411.jsonl
logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_temperature1.0_topP0.87_completions_20250620_110455_4025.jsonl
logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_full_temperature1.0_topP0.87_completions_20250620_110509_7186.jsonl 

Nxcode-CQ-7B-orpo
Num files: 3
logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/Nxcode-CQ-7B/Nxcode-CQ-7B-orpo_complete_task_prompt_full_temperature1.0_topP0.87_completions_20250621_030726_5547.jsonl
logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/Nxcode-CQ-7B/Nxcode-CQ-7B-orpo_complete_task_prompt_temperature1.0_topP0.87_completions_20250621_030

In [7]:
# convert to dict of dict with prompt types as keys
prompt_dict = dict()
for model_name, files in jsonl_files.items():
    result = dict()
    for file in files:
        if prompt_basic_key in file:
            result[prompt_basic_key] = file
        elif prompt_full_key in file:
            result[prompt_full_key] = file
        else:
            result[prompt_medium_key] = file       # the remaining file - neither full nor basic prompt
    #assert len(result)==3, f'One or more files are missing in {result}'
    prompt_dict[ model_name ] = result

for model_name, prompt_file_pair in prompt_dict.items():
    print(model_name)
    print(len(prompt_file_pair))
    for prompt, file in prompt_file_pair.items():
        print(f'\t{prompt}: {file}', '\n')

phixtral-2x2
3
	complete_task_prompt_basic: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_basic_temperature1.0_topP0.87_completions_20250620_110343_5411.jsonl 

	complete_task_prompt: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_temperature1.0_topP0.87_completions_20250620_110455_4025.jsonl 

	complete_task_prompt_full: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_full_temperature1.0_topP0.87_completions_20250620_110509_7186.jsonl 

Nxcode-CQ-7B-orpo
3
	complete_task_prompt_full: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/Nxcode-CQ-7B/Nxcode-CQ-7B-orpo_complete_task_prompt_full_temperature1.0_topP0.87_completions_20250621_030726_5547.jsonl 

	complete_task_prompt: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/Nxcode-CQ-7B/Nxcod

## 2. Clean and Evaluate Generated Code

```
:param mode:
       'human_eval' - using the HumanEval dataset with or without func header;
       'mbpp' - using the MBPP dataset;
       'lbpp' - using the LBPP dataset;
       'big_code' - using the Big Code Benchmark dataset.
:param k:
        [1] - pass@1 score
        [1,5] - pass@1 and pass@5 scores, etc.
```

In [8]:
final_res = defaultdict(list)
for model_name, prompt_file_pair in prompt_dict.items():
    print('\nModel:', model_name)
    
    # enforce the order of prompts    
    prompt_keys = [prompt_basic_key, prompt_medium_key, prompt_full_key]

    for prompt_key in prompt_keys:
        print('\nPrompt key:', prompt_key)

        # load file with results for cleaning
        file_name = prompt_file_pair[ prompt_key ]
        results   = list(stream_jsonl(file_name))
        print(f'\nFile: {file_name}\nNumber of completions: {len(results)}')

        # apply different degrees of cleaning
        results_cleaned_partially, results_cleaned_fully, results_cleaned_fully_light = [], [], []
        for item in results:
            task_id    = item['task_id']
            completion = item['completion']
            
            # no signature if lbpp and mbpp
            if dataset_name in ['lbpp', 'mbpp']:
                signature = None
            else:
                signature  = tasks[task_id]['prompt']
            completion_cleaned_partially   = get_code_in_fences(completion)
            completion_cleaned_fully       = clean_code(completion, signature)
            completion_cleaned_fully_light = clean_code_light(completion, signature)

            results_cleaned_partially.append(  {'task_id': task_id, 'completion': completion_cleaned_partially} )
            results_cleaned_fully.append(      {'task_id': task_id, 'completion': completion_cleaned_fully} )
            results_cleaned_fully_light.append( {'task_id': task_id, 'completion': completion_cleaned_fully_light} )
            
        # save cleaned files
        file_name_cleaned_partially   = file_name.replace('.jsonl', '') + '_cleaned_partially.jsonl'
        file_name_cleaned_fully       = file_name.replace('.jsonl', '') + '_cleaned_fully.jsonl'
        file_name_cleaned_fully_light = file_name.replace('.jsonl', '') + '_cleaned_fully_light.jsonl'

        write_jsonl(file_name_cleaned_partially, results_cleaned_partially)
        write_jsonl(file_name_cleaned_fully, results_cleaned_fully)
        write_jsonl(file_name_cleaned_fully_light, results_cleaned_fully_light)
        
        print('\nEVALUATING RAW FILE')
        pass1_score_raw                   = evaluate_functional_correctness(
            file_name,
            k=[1],
            mode=mode, )
        
        print('\nEVALUATING PARTIALLY CLEANED FILE')
        pass1_score_cleaned_partially     = evaluate_functional_correctness(
            file_name_cleaned_partially,
            k=[1],
            mode=mode, )
        
        print('\nEVALUATING FULLY CLEANED FILE')
        pass1_score_cleaned_fully          = evaluate_functional_correctness(
            file_name_cleaned_fully,
            k=[1],
            mode=mode, )
        
        print('\nEVALUATING FULLY CLEANED LIGHT FILE')
        pass1_score_cleaned_fully_light    = evaluate_functional_correctness(
            file_name_cleaned_fully_light,
            k=[1],
            mode=mode, )       
                
        # enforce the order of cleaning modes
        local_scores = [ pass1_score_raw['pass@1'], pass1_score_cleaned_partially['pass@1'],
                         pass1_score_cleaned_fully['pass@1'], pass1_score_cleaned_fully_light['pass@1'], ]        
        final_res[model_name].extend( local_scores )
        
        print('\nResults for:', file_name)
        print('\nRaw score')
        print('Partially cleaned score')
        print('Fully cleaned score')
        print('Fully light cleaned score\n')
        print(*local_scores, sep='\n')
        print('\n', '='*75, '\n', sep='')
    print('Interim  results:', *final_res.items(), sep='\n')
    print('\n', '='*100, '\n', sep='')


Model: phixtral-2x2

Prompt key: complete_task_prompt_basic

File: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_basic_temperature1.0_topP0.87_completions_20250620_110343_5411.jsonl
Number of completions: 162

EVALUATING RAW FILE
Reading samples...


162it [00:00, 16140.37it/s]


Running test suites...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162/162 [00:11<00:00, 13.56it/s]


Writing results to logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/phixtral-2x2_8/phixtral-2x2_8_complete_task_prompt_basic_temperature1.0_topP0.87_completions_20250620_110343_5411.jsonl_results.jsonl...


...

Running test suites...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162/162 [00:11<00:00, 14.65it/s]


Writing results to logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/OpenCodeInterpreter-DS-6.7B/OpenCodeInterpreter-DS-6.7B_complete_task_prompt_full_temperature1.0_topP0.87_completions_20250621_005626_7242_cleaned_fully_light.jsonl_results.jsonl...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 162/162 [00:00<00:00, 62480.67it/s]


Results for: logs_results/final-round-one-model-top-p_experiment/LBPP/top-p_087/OpenCodeInterpreter-DS-6.7B/OpenCodeInterpreter-DS-6.7B_complete_task_prompt_full_temperature1.0_topP0.87_completions_20250621_005626_7242.jsonl

Raw score
Partially cleaned score
Fully cleaned score
Fully light cleaned score

0.0
0.2716049382716049
0.29012345679012347
0.2839506172839506


Interim  results:
('phixtral-2x2', [0.024691358024691357, 0.12962962962962962, 0.12345679012345678, 0.12962962962962962, 0.012345679012345678, 0.12345679012345678, 0.12962962962962962, 0.13580246913580246, 0.012345679012345678, 0.08641975308641975, 0.11728395061728394, 0.12345679012345678])
('Nxcode-CQ-7B-orpo', [0.006172839506172839, 0.19753086419753085, 0.20987654320987653, 0.20987654320987653, 0.012345679012345678, 0.21604938271604937, 0.20987654320987653, 0.21604938271604937, 0.018518518518518517, 0.15432098765432098, 0.19753086419753085, 0.19753086419753085])
('Artigenz-Coder-DS-6.7B', [0.024691358024691357, 0.27777

In [9]:
cols   = [c for c in model_names if c in final_res]
df_res = pd.DataFrame(final_res)
df_res = df_res[cols]
results_file = 'results/indiv_model_results_lbpp_top-p_087_20250623.csv'
df_res.to_csv(results_file, index=False)
df_res

,phixtral-2x2,Solar-10.7B,Llama-3.1-8B,codegemma-7b-it,deepseek-coder-6.7b,OpenCodeInterpreter-DS-6.7B,Artigenz-Coder-DS-6.7B,CodeQwen1.5-7B-Chat,Nxcode-CQ-7B-orpo
0,0.024691,0.024691,0.000000,0.000000,0.006173,0.024691,0.024691,0.024691,0.006173
1,0.129630,0.055556,0.222222,0.117284,0.271605,0.277778,0.277778,0.197531,0.197531
2,0.123457,0.092593,0.179012,0.141975,0.271605,0.265432,0.234568,0.203704,0.209877
3,0.129630,0.092593,0.222222,0.148148,0.271605,0.277778,0.277778,0.203704,0.209877
4,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346
5,0.123457,0.049383,0.160494,0.111111,0.271605,0.265432,0.240741,0.209877,0.216049
6,0.129630,0.074074,0.191358,0.179012,0.259259,0.265432,0.228395,0.222222,0.209877
7,0.135802,0.074074,0.216049,0.179012,0.271605,0.265432,0.246914,0.222222,0.216049
8,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018519
9,0.086420,0.037037,0.203704,0.135802,0.216049,0.271605,0.240741,0.160494,0.154321


In [10]:
unknown_variable

NameError: name 'unknown_variable' is not defined

## 5. Debug errors

In [9]:
# load file with results for debugging
file_name = 'logs/final-round-one-model/BigCode/Nous-Hermes-2-Solar-10.7B/BigCode_Nous-Hermes-2-Solar-10.7B_complete_code_prompt_basic_temperature1.0_topP1.0_completions_20250402_030815_4464.jsonl'
results1 = list(stream_jsonl(file_name))
file_name = 'logs/final-round-one-model/BigCode/Nous-Hermes-2-Solar-10.7B/BigCode_Nous-Hermes-2-Solar-10.7B_complete_code_prompt_temperature1.0_topP1.0_completions_20250402_031004_3447.jsonl'
results2 = list(stream_jsonl(file_name))
len(results1), len(results2)

(500, 500)

In [12]:
# which key is missing
keys1 = [i['task_id'] for i in results1]
keys2 = [i['task_id'] for i in results2]
[key for key in keys2 if key not in keys1]

['BigCodeBench/1138']

In [16]:
one_res = { 'task_id': 'BigCodeBench/1138', 'completion': 'completion' }

In [17]:
results1.append( one_res )
len(results1)

500

In [15]:
results1[-1]['task_id'] = 'BigCodeBench/1138'
results1[-1]

{'task_id': 'BigCodeBench/1138', 'completion': 'completion'}

In [16]:
file_name = 'logs/final-round-one-model/BigCode/Nous-Hermes-2-Solar-10.7B/BigCode_Nous-Hermes-2-Solar-10.7B_complete_code_prompt_basic_temperature1.0_topP1.0_completions_20250402_030815_4464.jsonl'
write_jsonl(file_name, results1)

In [497]:
# review
idx = 55
for res in [results, results_cleaned_partially, results_cleaned_fully, results_cleaned_fully_light]:
    print(res[idx]['completion'])
    print('\n', '='*75, '\n', sep='')

def fib(n: int):
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

# Test cases
print(fib(10))  # Expected output: 34
print(fib(1))   # Expected output: 0
print(fib(8))   # Expected output: 13


def fib(n: int):
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

# Test cases
print(fib(10))  # Expected output: 34
print(fib(1))   # Expected output: 0
print(fib(8))   # Expected output: 13


from typing import *

def fib(n: int):
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in rang

In [ ]:
results[0]